## Import Libraries

In [2]:
from __future__ import division, print_function, absolute_import

import tflearn
from tflearn.data_utils import shuffle
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers. conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation

import pickle, random
import numpy as np
import matplotlib.pyplot as plt

#%matplotlib inline
np.random.seed(411)

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Load the Dataset

In [3]:
X_train, Y_train, X_test, Y_test = pickle.load(open("../data/full_dataset.pkl", "rb"), encoding='latin1')

In [ ]:
print("Training images:", X_train.shape)
print("Test images:", X_test.shape)

Training images: (56780, 32, 32, 3)
Test images: (15000, 32, 32, 3)


In [ ]:
plt.figure(figsize=(32,32)) 
amount = 10

for i in range(amount):
    rand_id = random.randint(1, X_train.shape[0])
    plt.subplot(1,amount,(i+1))
    plt.title("Ground Truth : {}".format(Y_train[rand_id]))
    plt.imshow(X_train[rand_id])

plt.show()

In [15]:
# Shuffle the data
X_train, Y_train = shuffle(X_train, Y_train)

# Make sure the data is normalized
img_prep = ImagePreprocessing()
img_prep.add_featurewise_zero_center()
img_prep.add_featurewise_stdnorm()

# Create extra synthetic training data by flipping, rotating and blurring the images on our dataset
img_aug = ImageAugmentation()
img_aug.add_random_flip_leftright()
img_aug.add_random_rotation(max_angle=25.)
img_aug.add_random_blur(sigma_max=3.)

In [16]:
# Input is a 32x32 imamge with 3 color channels (red, green and blue)
network = input_data(shape=[None, 32, 32, 3], 
                     data_preprocessing=img_prep, 
                     data_augmentation=img_aug)

# Step 1: Convolution
network = conv_2d(network, 32, 3, activation='relu')

# Step 2: Max pooling
network = max_pool_2d(network, 2)

# Step 3: Convolution again
network = conv_2d(network, 64, 3, activation='relu')

# Step 4: Convolution yet again
network = conv_2d(network, 64, 3, activation='relu')

# Step 5: Max pooling again
network = max_pool_2d(network, 2)

# Step 6: Fully-connected 512 node neural network
network = fully_connected(network, 512, activation='relu')

# Step 7: Dropout - throw away some data randomly during training to prevent over-fitting
network = dropout(network, 0.5)

# Step 8: Fully-connected neural network with two outputs (0=isn't a bird, 1=is a bird) to make the final prediction
network = fully_connected(network, 2, activation='softmax')

# Tell tflearn how we want to train the network
network = regression(network, optimizer='adam', 
                     loss='categorical_crossentropy', 
                     learning_rate=0.001)

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


## Train the Deep Network

In [17]:
# Wrap the network in a model object
model = tflearn.DNN(network, tensorboard_verbose=0, checkpoint_path='bird-classifier.tfl.ckpt')

In [18]:
import time

In [19]:
# Train it! We'll do 100 training passes and monitor it as it goes

start_time = time.time()
model.fit(X_train, Y_train, n_epoch=100, shuffle=True, validation_set=(X_test, Y_test),
          show_metric=True, batch_size=96,
          snapshot_epoch=True,
          run_id='bird-classifier')


print("--- Learning process takes %.4f seconds ---" % (time.time() - start_time))

Training Step: 4097  | total loss: 0.30057 | time: 270.439s
| Adam | epoch: 007 | loss: 0.30057 - acc: 0.8829 -- iter: 52320/56780


KeyboardInterrupt: 

## Save the model

In [ ]:
model.save("bird-classifier.tfl")
print("Network trained and saved as bird-classifier.tfl!")

<hr/>